In [ ]:
import pickle
import pandas as pd

# Replace 'your_file.pkl' with the actual path to your .pkl file
with open('../2. data/llegadas.pkl', 'rb') as file:
    data = pickle.load(file)



In [ ]:
# Pruebas Waiting list
wl = WL([1,2,3], [5,6,7,8])

for paciente in pacientes_prueba_1:
    if paciente.grd in wl.sub_listas[paciente.requerimiento_inicial]:
        wl.agregar_paciente(paciente)
        # print(f"Paciente {paciente.id} agregado a la sublista {paciente.requerimiento_inicial} - GRD {paciente.grd}")

print(wl)

wl.actualizar_tiempo()

print(wl)

In [ ]:
# Se instancian a todos los pacientes a partir de los datos de incertidumbre
pacientes = {}
lista_pacientes = []
for hospital in range(0,4): # 0 son llegadas a WL
    pacientes[hospital] = {}
    for requerimiento in range(1,4):
        pacientes[hospital][requerimiento] = {}
        for grd in range(1,9):
            pacientes[hospital][requerimiento][grd] = []
            cantidad_pacientes = len(incertidumbre[hospital][requerimiento][grd])
            if cantidad_pacientes != 0:
                for i in range(cantidad_pacientes):
                    paciente = Paciente(hospital, requerimiento, grd, incertidumbre[hospital][requerimiento][grd][i]) # i es el index de la lista
                    pacientes[hospital][requerimiento][grd].append(paciente)
                    lista_pacientes.append(paciente)
            else:
                pacientes[hospital][requerimiento][grd] = []

# Se generan tantas listas como pacientes haya
def agrupar_pacientes_por_llegada(pacientes):
    pacientes_separados_por_llegada = {}
    for paciente in pacientes:
        dia = paciente.ti_inicial
        if dia not in pacientes_separados_por_llegada:
            pacientes_separados_por_llegada[dia] = []
        pacientes_separados_por_llegada[dia].append(paciente)
    return pacientes_separados_por_llegada

pacientes_separados_por_llegada = agrupar_pacientes_por_llegada(lista_pacientes)
pacientes_prueba_1 = pacientes_separados_por_llegada.get(2).copy()
pacientes_prueba_2 = pacientes_separados_por_llegada.get(3).copy()

wl = WL([1,2,3], [1,2,3,4,5,6,7,8])
ps = PS([1, 2, 3], [1, 2, 3, 4, 5, 6, 7, 8])
end = End([1, 2, 3], [1, 2, 3, 4, 5, 6, 7, 8])
h_1 = Hospital(1)
h_2 = Hospital(2)
h_1.ocupacion()

for i in range(5):
    for j in range(1, 6):
        h_1.agregar_paciente(pacientes_prueba_1.pop(), j)

for i in range(5):
    for j in range(1, 6):
        h_2.agregar_paciente(pacientes_prueba_2.pop(), j)

pacientes_ed_1 = h_1.ED.pacientes.copy()
pacientes_ed_2 = h_2.ED.pacientes.copy()

h_1.actualizar_tiempo()
for paciente in pacientes_ed_1:
    h_1.sacar_paciente(paciente, p.dict_unidades["ED"])
    ps.agregar_paciente(paciente)

h_2.actualizar_tiempo()
for paciente in pacientes_ed_2:
    h_2.sacar_paciente(paciente, p.dict_unidades["ED"])
    ps.agregar_paciente(paciente)

# print(ps)

for requerimiento in ps.sub_listas:
    for grd in ps.sub_listas[requerimiento]:
        if len(ps.sub_listas[requerimiento][grd]) > 0:
            for paciente in ps.sub_listas[requerimiento][grd]:
                for log in paciente.log_eventos:
                    print(log)
                print("----------------------")




In [ ]:
# Flujo para hacer que pacientes pasen por todas las unidades que necesitan


def imprimir_log(log_eventos: list, ultimo = False):
    if not ultimo:
        for evento in log_eventos:
            print(evento)
    else:
        print(log_eventos[-1])

paciente_ed_1 = None
paciente_ed_2 = None
paciente_ed_3 = None
paciente_wl = None
for ciclo in range(1, 4):
    for paciente in pacientes_separados_por_llegada[ciclo]:
        if paciente.unidad_actual == p.dict_unidades["ED"] and paciente.hospital_actual == 1:
            paciente_ed_1 = paciente
        elif paciente.unidad_actual == p.dict_unidades["ED"] and paciente.hospital_actual == 2:
            paciente_ed_2 = paciente
        elif paciente.unidad_actual == p.dict_unidades["ED"] and paciente.hospital_actual == 3:
            paciente_ed_3 = paciente
        elif paciente.unidad_actual == p.dict_unidades["WL"]:
            paciente_wl = paciente
        if paciente_ed_1 and paciente_ed_2 and paciente_ed_3 and paciente_wl:
            break

# 3 | {1: [1, 2, 3]} | {1: [1, 3, 12]}
paciente = paciente_wl
print(paciente.ti_inicial, "|",paciente.camino, "|", paciente.espera)
print(paciente)

paciente.cambiar_unidad(p.dict_hospitales["Hospital_1"], p.dict_unidades["GA"])
imprimir_log(paciente.log_eventos, ultimo = True)

paciente.cambiar_unidad(paciente.hospital_actual, paciente.unidad_requerida)
imprimir_log(paciente.log_eventos, ultimo = True)

for unidades in range(len(paciente.camino[paciente.hospital_actual]) - 1): # la ultima la ve el while
    for i in range(paciente.espera[paciente.hospital_actual][0]):
        paciente.actualizar_tiempo()
    # paciente.actualizar_tiempo() # espera 1 cada vez
    paciente.cambiar_unidad(paciente.hospital_actual, paciente.unidad_requerida)
    imprimir_log(paciente.log_eventos, ultimo = True)

while paciente.unidad_requerida != p.dict_unidades["END"]:
    paciente.actualizar_tiempo()
paciente.cambiar_unidad(paciente.hospital_actual, p.dict_unidades["END"])
imprimir_log(paciente.log_eventos, ultimo = True)
paciente.costo_social

In [ ]:
print(h_1)

for evento in h_1.ED.pacientes[0].log_eventos:
    print(evento)
h_1.actualizar_tiempo()
paciente = h_1.sacar_paciente(h_1.ED.pacientes[0], p.dict_unidades["ED"])

h_1.agregar_paciente(paciente, p.dict_unidades["OR"])

print("----------------------")

for evento in paciente.log_eventos:
    print(evento)

h_1.actualizar_tiempo()
paciente = h_1.sacar_paciente(paciente, p.dict_unidades["OR"])
print(paciente)
h_1.agregar_paciente(paciente, p.dict_unidades["ICU"])
print("----------------------")
for evento in paciente.log_eventos:
    print(evento)

h_1.actualizar_tiempo()
paciente = h_1.sacar_paciente(paciente, p.dict_unidades["ICU"])

h_1.agregar_paciente(paciente, p.dict_unidades["SDU/WARD"])
print("----------------------")
for evento in paciente.log_eventos:
    print(evento)

print(h_1)

h_1.actualizar_tiempo()
paciente = h_1.sacar_paciente(paciente, p.dict_unidades["SDU/WARD"])

end.agregar_paciente(paciente)
print("----------------------")
for evento in paciente.log_eventos:
    print(evento)

pa_2 = h_2.sacar_paciente(h_2.SDU_WARD.pacientes[0], p.dict_unidades["SDU/WARD"])
print(pa_2)
end.agregar_paciente(pa_2)
print("----------------------")


for paciente in end.sub_listas[1][4]:
    print(paciente.unidad_actual, paciente)

print(h_2.ocupacion())

In [ ]:
# Testeo x para ver si funcionan las unidades, si funcionaron
# Se instancian a todos los pacientes a partir de los datos de incertidumbre
pacientes = {}
lista_pacientes = []
for hospital in range(0,4): # 0 son llegadas a WL
    pacientes[hospital] = {}
    for requerimiento in range(1,4):
        pacientes[hospital][requerimiento] = {}
        for grd in range(1,9):
            pacientes[hospital][requerimiento][grd] = []
            cantidad_pacientes = len(incertidumbre[hospital][requerimiento][grd])
            if cantidad_pacientes != 0:
                for i in range(cantidad_pacientes):
                    paciente = Paciente(hospital, requerimiento, grd, incertidumbre[hospital][requerimiento][grd][i]) # i es el index de la lista
                    pacientes[hospital][requerimiento][grd].append(paciente)
                    lista_pacientes.append(paciente)
            else:
                pacientes[hospital][requerimiento][grd] = []

# Se generan tantas listas como pacientes haya
def agrupar_pacientes_por_llegada(pacientes):
    pacientes_separados_por_llegada = {}
    for paciente in pacientes:
        dia = paciente.ti_inicial
        if dia not in pacientes_separados_por_llegada:
            pacientes_separados_por_llegada[dia] = []
        pacientes_separados_por_llegada[dia].append(paciente)
    return pacientes_separados_por_llegada

pacientes_separados_por_llegada = agrupar_pacientes_por_llegada(lista_pacientes)

pacientes_prueba = pacientes_separados_por_llegada.get(2)

# Prueba unidades
or_1 = Or(1)
ed_1 = Ed(1)
icu_1 = Icu(1)
sdu_ward_1 = SduWard(1)
ga_1 = Ga(1)
termino = Termino([1, 2, 3], [1, 2, 3, 4, 5, 6, 7, 8])
print(or_1)
for paciente in pacientes_prueba[0:4]:
    or_1.agregar_paciente(paciente)
print(or_1)

# print(pacientes_prueba[0].log_eventos)
# pacientes_prueba[0].actualizar_tiempo()
# or_1.agregar_paciente(pacientes_prueba[0])
# print(pacientes_prueba[0].log_eventos)
# pacientes_prueba[0].actualizar_tiempo()
# print(or_1.sacar_paciente(pacientes_prueba[0]))
# print(or_1.sacar_paciente(pacientes_prueba[0]))
# print(or_1.sacar_paciente(pacientes_prueba[0]))
# print(or_1.agregar_paciente(pacientes_prueba[0]))
# print(or_1.agregar_paciente(pacientes_prueba[0]))
# ed_1.agregar_paciente(paciente_sacado)
# print(pacientes_prueba[0].log_eventos)
# pacientes_prueba[0].actualizar_tiempo()
# icu_1.agregar_paciente(pacientes_prueba[0])
# print(pacientes_prueba[0].log_eventos)
# pacientes_prueba[0].actualizar_tiempo()
# ga_1.agregar_paciente(pacientes_prueba[0])
# print(pacientes_prueba[0].log_eventos)
# pacientes_prueba[0].actualizar_tiempo()
# sdu_ward_1.agregar_paciente(pacientes_prueba[0])
# print(pacientes_prueba[0].log_eventos)
# termino.agregar_paciente(pacientes_prueba[0])
# print(pacientes_prueba[0].log_eventos)

   

In [ ]:
def ajuste_kde(dato: list, ocurrencias: list, bandwidth: float = 1, pvalue: float = 0.06, plot: bool = True, titulo_plot: str = "Ajuste KDE"):    
    ###################### Datos a analizar ######################
    dato = np.array(dato) # Aca no se divide por 12 ya que es cantidad de llegadas por dia
    ocurrencias = np.array(ocurrencias)
    raw_data = np.repeat(dato, ocurrencias)
    ##############################################################
    pvalue_obtenido = 0
    while pvalue_obtenido < pvalue:
        # Estimador del kernerl density discreto #####################
        kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth).fit(raw_data[:, None])
        # Calculo el valor p de la prueba de chi2
        metricas, kde_pdf = calculate_kde_metrics(dato, ocurrencias, kde)
        pvalue_obtenido = metricas["Chi2"]["p-value"]
        # Si el pvalue obtenido es menor al pvalue deseado, disminuyo el ancho de banda
        if pvalue_obtenido < pvalue:
            # Disminuyo el ancho de banda
            bandwidth -= 0.005
            # Si el ancho de banda es menor a 0.01, salgo del while
            if bandwidth < 0.01:
                print("No se encontró un ancho de banda adecuado sobre 0.5")
                break
        ##############################################################

    if plot:
        ###################### Plot ##################################
        # Histograma de los datos, tantos bins como el valor maximo de los datos
        plt.figure(figsize=(10, 3)) # tamaño grafico
        plt.bar(dato, ocurrencias / ocurrencias.sum(), alpha=0.6, color='g', edgecolor='black', label='Bar Plot')

        # pdf del KDE ajustado a los datos
        x_vals = np.linspace(min(raw_data), max(raw_data), 1000)
        log_dens = kde.score_samples(x_vals[:, None])
        kde_pdf = np.exp(log_dens)
        plt.plot(x_vals, kde_pdf, color='black', label=f'KDE (Bandwidth = {round(bandwidth,2)})')
        # puntos de las intersecciones
        x_vals_puntos = np.linspace(int(min(raw_data)), int(max(raw_data)), int(max(raw_data)) - int(min(raw_data)) + 1)
        log_dens_puntos = kde.score_samples(x_vals_puntos[:, None])
        kde_pdf_puntos = np.exp(log_dens_puntos)
        plt.scatter(x_vals_puntos, kde_pdf_puntos, color='red', label=f'KDE intersecciones')


        # Leyendas y nombres
        plt.xlabel('LOS')
        plt.ylabel('Densidad')
        plt.title(titulo_plot)
        plt.legend()
        plt.grid(True)
        plt.show()
        ##############################################################
    
    return kde, kde_pdf, metricas, bandwidth

In [8]:
data["WL"][1]

{(1, 1): 0,
 (1, 2): 0,
 (1, 3): 0,
 (2, 1): 0,
 (2, 2): 0,
 (2, 3): 0,
 (3, 1): 0,
 (3, 2): 0,
 (3, 3): 0,
 (4, 1): 0,
 (4, 2): 0,
 (4, 3): 0,
 (5, 1): 0,
 (5, 2): 2,
 (5, 3): 4,
 (6, 1): 2,
 (6, 2): 2,
 (6, 3): 0,
 (7, 1): 1,
 (7, 2): 1,
 (7, 3): 2,
 (8, 1): 0,
 (8, 2): 0,
 (8, 3): 2}

In [ ]:
###################### Distintos tipos de bandwith para ir probando
dict_bandwith = {
    'normal_mean': ((4/(3*len(raw_data)))**(1/5)) * np.std(raw_data),
    'normal_mean_iqr': ((4/(3*len(raw_data)))**(1/5)) * (np.percentile(raw_data, 75) - np.percentile(raw_data, 25)),
    "scott": 1.06 * np.std(raw_data) * (len(raw_data) ** (-1 / 5))
}
##############################################################

In [ ]:
# ejemplo internet
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from scipy.stats import norm, lognorm, gamma, poisson, chisquare, kstest, pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

rand_seed = 100

def make_data_normal(data_count=100):
    np.random.seed(rand_seed)
    x = np.random.normal(0, 1, data_count)
    dist = lambda z: stats.norm(0, 1).pdf(z)
    return x, dist

def make_data_binormal(data_count=100):
    alpha = 0.3
    np.random.seed(rand_seed)
    x = np.concatenate([
        np.random.normal(-1, 2, int(data_count * alpha)),
        np.random.normal(5, 1, int(data_count * (1 - alpha)))
    ])
    dist = lambda z: alpha * stats.norm(-1, 2).pdf(z) + (1 - alpha) * stats.norm(5, 1).pdf(z)
    return x, dist

def make_data_exp(data_count=100):
    alpha = 0.3
    np.random.seed(rand_seed)
    x = np.concatenate([
        np.random.exponential(1, int(data_count * alpha)),
        np.random.exponential(1, int(data_count * (1 - alpha))) + 1
    ])
    dist = lambda z: alpha * stats.expon(0).pdf(z) + (1 - alpha) * stats.expon(1).pdf(z)
    return x, dist

def make_data_uniform(data_count=100):
    alpha = 0.3
    np.random.seed(rand_seed)
    x = np.concatenate([
        np.random.uniform(-1, 1, int(data_count * alpha)),
        np.random.uniform(0, 1, int(data_count * (1 - alpha)))
    ])
    dist = lambda z: alpha * stats.uniform(-1, 1).pdf(z) + (1 - alpha) * stats.uniform(0, 1).pdf(z)
    return x, dist

x_norm, dist_norm = make_data_normal()
x_binorm, dist_binorm = make_data_binormal()
x_exp, dist_exp = make_data_exp()
x_uni, dist_uni = make_data_uniform()

fig, ax = plt.subplots(1, 4, figsize=(12, 3))
names = ['Normal', 'Bi-normal', 'Exponential', 'Bi-Uniform']
for i, d in enumerate([dist_norm, dist_binorm, dist_exp, dist_uni]):
    x = np.linspace(-8, 8, 100)
    ax[i].fill(x, d(x), color='C0', alpha=0.5)
    ax[i].set_ylim(0, 1)
    ax[i].set_xlim(-8, 8)
    ax[i].set_xlabel('x')
    ax[i].set_ylabel('p(x)')
    ax[i].set_title(names[i])
fig.tight_layout()
fig.show()

def kernel(k: str):
    """Kernel Functions.
    Ref: https://en.wikipedia.org/wiki/Kernel_(statistics)

    Args:
        k (str): Kernel name. Can be one of ['gaussian', 'epanechnikov', 'cosine', 'linear'.]
    """
    
    if k not in ['gaussian', 'epanechnikov', 'cosine', 'linear']:
        raise ValueError('Unknown kernel.')

    def bounded(f):
        def _f(x):
            return f(x) if np.abs(x) <= 1 else 0
        return _f

    if k == 'gaussian':
        return lambda u: 1 / np.sqrt(2 * np.pi) * np.exp(-1 / 2 * u * u)
    elif k == 'epanechnikov':
        return bounded(lambda u: (3 / 4 * (1 - u * u)))
    elif k =='cosine':
        return bounded(lambda u: np.pi / 4 * np.cos(np.pi / 2 * u))
    elif k == 'linear':
        return bounded(lambda u: 1 - np.abs(u))
    
def bw_scott(data: np.ndarray):
    std_dev = np.std(data, axis=0, ddof=1)
    n = len(data)
    return 3.49 * std_dev * n ** (-0.333)

def bw_silverman(data: np.ndarray):
    def _select_sigma(x):
        normalizer = 1.349
        iqr = (stats.scoreatpercentile(x, 75) - stats.scoreatpercentile(x, 25)) / normalizer
        std_dev = np.std(x, axis=0, ddof=1)
        return np.minimum(std_dev, iqr) if iqr > 0 else std_dev
    sigma = _select_sigma(data)
    n = len(data)
    return 0.9 * sigma * n ** (-0.2)

def bw_mlcv(data: np.ndarray, k):
    """
    Ref: https://rdrr.io/cran/kedd/src/R/MLCV.R
    """
    n = len(data)
    x = np.linspace(np.min(data), np.max(data), n)
    def mlcv(h):
        fj = np.zeros(n)
        for j in range(n):
            for i in range(n):
                if i == j: continue
                fj[j] += k((x[j] - data[i]) / h)
            fj[j] /= (n - 1) * h
        return -np.mean(np.log(fj[fj > 0]))
    h = optimize.minimize(mlcv, 1)
    if np.abs(h.x[0]) > 10:
        return bw_scott(data)
    return h.x[0]

def kde(data, k=None, h=None, x=None):
    """Kernel Density Estimation.

    Args:
        data (np.ndarray): Data.
        k (function): Kernel function.
        h (float): Bandwidth.
        x (np.ndarray, optional): Grid. Defaults to None.

    Returns:
        np.ndarray: Kernel density estimation.
    """
    if x is None:
        x = np.linspace(np.min(data), np.max(data), 1000)
    if h is None:
        h = bw_silverman(data)
    if k is None:
        k = kernel('gaussian')
    n = len(data)
    kde = np.zeros_like(x)
    for j in range(len(x)):
        for i in range(n):
            kde[j] += k((x[j] - data[i]) / h)
        kde[j] /= n * h
    return kde

data = [
    ('Normal', make_data_normal),
    ('Bimodal (Normal)', make_data_binormal),
    ('Bimodal (Exp)', make_data_exp),
    ('Bimodal (Uniform)', make_data_uniform)
]
kernels = [
    ('Gaussian', kernel('gaussian')),
    ('Epanechnikov', kernel('epanechnikov')),
    ('Cosine', kernel('cosine')),
    ('Linear', kernel('linear'))
]
bw_algorithms = [
    ('Scott', bw_scott),
    ('Silverman', bw_silverman),
    ('MLCV', bw_mlcv),
]
mses = []

def run_kde(ax, data, kernel):
    x, dist = data[1]()
    x_plot = np.linspace(np.min(x) * 1.05, np.max(x) * 1.05, 1000)
    ax.grid(True)
    ax.fill_between(x_plot, dist(x_plot), fc='silver', alpha=0.5)
    ax.plot(x, np.full_like(x, -0.02), '|k', markeredgewidth=1)
    ax.hist(x, density=True, alpha=0.2, bins=20, rwidth=0.9)
    for bw in bw_algorithms:
        if bw[0] == 'MLCV':
            h = bw[1](x, kernel[1])
        else:
            h = bw[1](x)
        x_kde = kde(x, kernel[1], h=h, x=x_plot)
        mse = np.mean((dist(x_plot) - x_kde) ** 2)
        mses.append({
            'data': data[0],
            'kernel': kernel[0],
            'bw_algorithm': bw[0],
            'h': round(h, 5),
            'mse': round(mse * 1000, 5), # To make differences more noticable
        })
        ax.plot(x_plot, x_kde, linewidth=1, label='$h_{\mathrm{' + bw[0] + '}} = ' + str(round(h, 5)) + '$')
    ax.legend(loc='best', fontsize='small')
    ax.set_title(f'{data[0]}, {kernel[0]}')

fig, axs = plt.subplots(len(data), len(kernels), figsize=(16, 12))

for i, d in enumerate(data):
    for j, k in enumerate(kernels):
        run_kde(axs[i, j], d, k)
    for bw in bw_algorithms:
        avg_h = np.mean([m['h'] for m in mses if m['data'] == d[0] and m['bw_algorithm'] == bw[0]])
        avg_mse = np.mean([m['mse'] for m in mses if m['data'] == d[0] and m['bw_algorithm'] == bw[0]])
        mses.append({
            'data': d[0],
            'kernel': '-',
            'bw_algorithm': bw[0],
            'h': round(avg_h, 5),
            'mse': round(avg_mse, 5),
        })

fig.tight_layout()
fig.show()
fig.savefig('eval.pdf')
pd.DataFrame(mses).to_csv('eval.csv', index=False)


In [ ]:
for i in range(1,4):
    for grd in range(1,9):
        for unidad in ["ICU", "SDU_WARD"]:
            # ---- 1. Filter your dataset
            tl_u = tl[tl["UNIDAD"].isin(["ICU", "OR", "SDU_WARD"])]
            v1 = tl_u[(tl_u["UNIDAD"] == unidad) & (tl_u["MS_GRD"] == grd) & (tl_u["HOSPITAL"] == f"Hospital_{i}")]
            # v1 = tl_u[(tl_u["UNIDAD"] == unidad) & (tl_u["MS_GRD"] == grd)]

            # ---- 2. Get LOS value counts
            vector = v1["LOS"].value_counts().reset_index().sort_values(by="LOS")
            vector.columns = ["LOS", "count"]

            # ---- 3. Build raw data from LOS values and their frequencies
            los = np.array(vector["LOS"])
            ocurrencias = np.array(vector["count"])
            raw_data = np.repeat(los, ocurrencias)


            # ---- 4. Definir los bins de 12 horas
            bin_width = 12
            max_los = np.max(raw_data)
            edges = np.arange(0, max_los + bin_width, bin_width)
            midpoints = (edges[:-1] + edges[1:]) / 2  # midpoint of (a, b] is (a + b) / 2
    

            # ---- 5. Meter datos a bins individuales (0 - 12], (12 - 24], ...)
            # agrega 0 si no hay datos para ese bin
            bin_indices = np.digitize(raw_data, edges, right=True)
            hist = np.array([(bin_indices == i).sum() for i in range(1, len(edges))])

#--------------------------------Hasta aca esta bien----------------------------------------

            # ---- 6. Fit log-normal distribution to the raw data
            shape, loc, scale = stats.lognorm.fit(raw_data, floc=0)



            # # ---- Optimización
            # def chi2_objective(params, hist, edges, total_count):
            #     shape, loc, scale = params
            #     # Ensure parameters are valid
            #     if shape <= 0 or scale <= 0:
            #         return np.inf
            #     cdf_low = stats.lognorm.cdf(edges[:-1], shape, loc, scale)
            #     cdf_high = stats.lognorm.cdf(edges[1:], shape, loc, scale)
            #     expected_probs = cdf_high - cdf_low
            #     expected_counts = expected_probs * total_count

            #     # Mask for bins with enough expected values
            #     mask = expected_counts >= 5
            #     if not np.any(mask):
            #         return np.inf
            #     obs = hist[mask]
            #     exp = expected_counts[mask]
                
            #     # Normalize expected to match sum of obs
            #     exp *= obs.sum() / exp.sum()

            #     chi2 = np.sum((obs - exp)**2 / exp)
            #     return chi2
            # initial_shape, initial_loc, initial_scale = stats.lognorm.fit(raw_data, floc=0)
            # # Minimize the chi-square
            # result = minimize(
            #     chi2_objective,
            #     x0=[initial_shape, initial_loc, initial_scale],
            #     args=(hist, edges, len(raw_data)),
            #     bounds=[(1e-5, None), (0, None), (1e-5, None)],
            #     method='L-BFGS-B'
            # )
            # # Extract optimized parameters
            # opt_shape, opt_loc, opt_scale = result.x
            # # print(f"Optimized shape={opt_shape:.4f}, loc={opt_loc:.4f}, scale={opt_scale:.2f}")
            # shape = opt_shape
            # loc = opt_loc
            # scale = opt_scale


            # # ---- 7. Compute expected counts per bin using log-normal CDF
            # cdf_low = stats.lognorm.cdf(edges[:-1], shape, loc, scale)
            # cdf_high = stats.lognorm.cdf(edges[1:], shape, loc, scale)
            # expected_probs = cdf_high - cdf_low
            # # Acumulo todo lo que queda de la cola en el ultimo bin
            # # expected_probs[-1] = 1 - (sum(expected_probs) - expected_probs[-1])
            # expected_counts = expected_probs * len(raw_data)


            # # ---- 9. Chi-square test (only where expected counts ≥ 5)
            # mask = expected_counts >= 5
            # obs = hist[mask]
            # exp = expected_counts[mask]

            # # display(obs)
            # # display(exp)

            # # Normalize expected counts to match the total of observed
            # if exp.sum() > 0:
            #     exp *= obs.sum() / exp.sum()

            # # Chi-square test
            # if np.any(mask):
            #     chi2, p = stats.chisquare(f_obs=obs, f_exp=exp)
            #     if p > 0.05:
            #         print(f"Chi² = {chi2:.2f}, p-value = {p:.10f}, Unidad={unidad}, Hospital={i}, GRD={grd}")
                    
            #         # ---- 8. Plot observed vs. expected
            #         plt.figure(figsize=(12, 6))
            #         plt.bar(midpoints, hist, width=bin_width - 2, alpha=0.6, label='Observed (12h bins)')
            #         plt.plot(midpoints, expected_counts, 'r--o', label='Log-normal expected')
            #         plt.xlabel('Length of Stay (hours)')
            #         plt.ylabel('Frequency')
            #         plt.title(f'LOS Histogram vs Log-normal Fit\nUnidad={unidad}, Hospital={i}, GRD={grd}')
            #         plt.grid(True)

            #         # Curva continua
            #         x_vals = np.linspace(1, max_los, 500)
            #         pdf_vals = stats.lognorm.pdf(x_vals, s=shape, loc=loc, scale=scale)
            #         pdf_scaled = pdf_vals * len(raw_data) * bin_width
            #         plt.plot(x_vals, pdf_scaled, 'g-', label='Log-normal PDF (smoothed)')

            #         plt.legend()
            #         plt.show()
            #         break

                    
            # else:
            #     print("Chi-square test skipped: not enough expected counts ≥ 5.")


In [ ]:
# Tabla para los OR

tabla = {
    1: [["MS_GRD", "LOS = 12", "LOS = 24"]],
    2: [["MS_GRD", "LOS = 12", "LOS = 24"]],
    3: [["MS_GRD", "LOS = 12", "LOS = 24"]]
}

for i in range(1,4):
    for grd in range(1,9):
        for unidad in ["OR"]:
            # ---- 1. Filter your dataset
            tl_u = tl[tl["UNIDAD"].isin(["ICU", "OR", "SDU_WARD"])]
            v1 = tl_u[(tl_u["UNIDAD"] == unidad) & (tl_u["MS_GRD"] == grd) & (tl_u["HOSPITAL"] == f"Hospital_{i}")]
            # v1 = tl_u[(tl_u["UNIDAD"] == unidad) & (tl_u["MS_GRD"] == grd)]

            # ---- 2. Get LOS value counts
            vector = v1["LOS"].value_counts().reset_index().sort_values(by="LOS")
            vector["%"] = vector["count"] / vector["count"].sum()
            vector1 = vector[["LOS", "%"]].reset_index()

            los_12 = 0
            los_24 = 0
            for index, row in vector1.iterrows():
                if row["LOS"] == 12:
                    los_12 = row["%"]
                elif row["LOS"] == 24:
                    los_24 = row["%"]

            tabla[i].append([grd, round(los_12,5), round(los_24,5)])
                    
            

    display(tabla[i])



In [ ]:
# Pasar a tablas de latex
for key, h in tabla.items():

    texto = f"""
\\begin{{table}}[H]
    \\centering
    \\begin{{tabular}}{{ccc}}
        \\toprule
        MS\_GRD & LOS = 12 & LOS = 24 \\\\
        \\midrule
        {h[1][0]} & {h[1][1]} & {h[1][2]} \\\\
        {h[2][0]} & {h[2][1]} & {h[2][2]} \\\\
        {h[3][0]} & {h[3][1]} & {h[3][2]} \\\\
        {h[4][0]} & {h[4][1]} & {h[4][2]} \\\\
        {h[5][0]} & {h[5][1]} & {h[5][2]} \\\\
        {h[6][0]} & {h[6][1]} & {h[6][2]} \\\\
        {h[7][0]} & {h[7][1]} & {h[7][2]} \\\\
        {h[8][0]} & {h[8][1]} & {h[8][2]} \\\\
        \\bottomrule
    \\end{{tabular}}
    \\caption{{Probabilidad LOS en OR para Hospital: {key}}}
    \\label{{tab:Probabilidad LOS en OR para Hospital: {key}}}
\\end{{table}}
    """

    print(texto)

In [ ]:
# Segundo intento (quedo pesimo)

# for i in range(1,4):
# for grd in range(1,9):
# for unidad in ["ICU", "SDU_WARD"]:

unidad = "ICU"  # Cambiar unidad
grd = 1

# ---- 1. Filtra los datos para obtener LOS y cantidad de ocurrencias
tl_u = tl[tl["UNIDAD"].isin(["ICU", "OR", "SDU_WARD"])]
# v1 = tl_u[(tl_u["UNIDAD"] == unidad) & (tl_u["MS_GRD"] == grd) & (tl_u["HOSPITAL"] == f"Hospital_{i}")]
v1 = tl_u[(tl_u["UNIDAD"] == unidad) & (tl_u["MS_GRD"] == grd)]

# ---- 2. Calcular la cantidad de veces que se repite ese LOS
vector = v1["LOS"].value_counts().reset_index().sort_values(by="LOS")
vector.columns = ["LOS", "count"]

# ---- 3. Pasarlo a formato para histograma
los = np.array(vector["LOS"])
ocurrencias = np.array(vector["count"])
raw_data = np.repeat(los, ocurrencias) # multiplica LOS por cantidad de repeticiones

# ---- 4. Definir los bins de 12 horas (para ajustar la curva)
bin_width = 12
max_los = np.max(raw_data) # Elige el maximo de LOS
edges = np.arange(0, max_los + bin_width, bin_width) # Define los bins
midpoints = (edges[:-1] + edges[1:]) / 2  # Define puntos medios de los bins

# ---- 5. Meter datos a bins individuales (0 - 12], (12 - 24], ...) agrega 0 si no hay datos para ese bin
bin_indices = np.digitize(raw_data, edges, right=True)
hist = np.array([(bin_indices == i).sum() for i in range(1, len(edges))])
bin_edges = edges
bin_upper_bounds = bin_edges[1:]

# ---- 6. Ajustar distribución a los datos en bins de 12 horas
shape, loc, scale = stats.lognorm.fit(raw_data, floc = 0)

# # Grafico para la curva suave log_normal
# x = np.linspace(min(bin_upper_bounds), max(bin_upper_bounds), 1000)
# y = stats.lognorm.pdf(x, shape, loc, scale) * len(raw_data) * bin_width
# plt.hist(raw_data, bins=bin_edges, alpha=0.6, color='g', label="Histogram")
# plt.plot(x, y, label="Fitted Log-normal", color='r')
# plt.xlabel('Value')
# plt.ylabel('Density')
# plt.legend()
# plt.show()

# Define interval of interest
  # for example
N = len(raw_data)
maximo = int(raw_data[-1])
maximo = int(maximo/12)
expected = []
for i in range(0, maximo):
    a = i * 12
    b = (i + 1) * 12
    p_interval = stats.lognorm.cdf(b, shape, loc, scale) - stats.lognorm.cdf(a, shape, loc, scale)
    expected_occurrences = N * p_interval
    expected.append(expected_occurrences)
esperado = np.array(expected)
observado = hist

# Test de chi cuadrado
mask = esperado >= 5

obs = observado[mask]
exp = esperado[mask]

exp *= obs.sum() / exp.sum()

chi2, p = stats.chisquare(f_obs=obs, f_exp=exp)
print(f"chi2 = {chi2}, p = {p}")

In [ ]:
los = [12, 24, 36, 48, 60, 72]
count = [5, 12, 8, 6, 4, 2]

def continuous(los, count):
    nueva_lista = []
    lista_cuartos = []
    for i in range(len(los)):
        # for j in range(count[i]):
        #     nueva_lista.append(los[i])
        nueva_lista.append(round((count[i])/4, 2))
    for cuarto in nueva_lista:
        lista_cuartos.append(cuarto)
        lista_cuartos.append(cuarto * 2)
        lista_cuartos.append(cuarto)
    
    uniones = []
    for i in range(0, len(lista_cuartos), 3):
        if i == 0:
            uniones.append(lista_cuartos[i])
            uniones.append(lista_cuartos[i + 1])
        elif i == len(lista_cuartos) - 3:
            uniones.append(lista_cuartos[i] + lista_cuartos[i - 1])
            uniones.append(lista_cuartos[i + 1])
            uniones.append(lista_cuartos[i + 2])
        else:
            uniones.append(lista_cuartos[i] + lista_cuartos[i - 1])
            uniones.append(lista_cuartos[i + 1])
    
    count = uniones.copy()

    new_los = []
    for i in range(len(uniones)):
        if i == 0:
            new_los.append(6)
        elif i == len(uniones) - 1:
            ultimo = new_los[-1]
            new_los.append(ultimo + 6)
        else:
            nuevo = new_los[-1] + 6
            new_los.append(nuevo)
    
    return new_los, count

uniones, count = continuous(los, count)

print(uniones)
print(count)




In [ ]:
# Parametros para cuando sea parte de una función
plot = True
bandwidth = 0.82
###################### Datos a analizar ######################
tl_u = tl[tl["UNIDAD"].isin(["ICU", "OR", "SDU_WARD"])]
v1 = tl_u[(tl_u["UNIDAD"] == "SDU_WARD") & (tl_u["MS_GRD"] == 1) & (tl_u["HOSPITAL"] == f"Hospital_{1}")]
vector = v1["LOS"].value_counts().reset_index().sort_values(by="LOS")
vector.columns = ["LOS", "count"]
los = np.array(vector["LOS"])/12 
ocurrencias = np.array(vector["count"])
raw_data = np.repeat(los, ocurrencias)
##############################################################

# Estimador del kernerl density discreto #####################
kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth).fit(raw_data[:, None])
##############################################################

if plot:
    ###################### Plot ##################################
    # Histograma de los datos, tantos bins como el valor maximo de los datos
    plt.figure(figsize=(10, 3)) # tamaño grafico
    plt.hist(raw_data, bins=int(raw_data[-1] - (raw_data[0] - 1)), density=True, alpha=0.6, color='g', edgecolor='black', label='Histogram')

    # pdf del KDE ajustado a los datos
    x_vals = np.linspace(min(raw_data), max(raw_data), 1000)
    log_dens = kde.score_samples(x_vals[:, None])
    kde_pdf = np.exp(log_dens)
    plt.plot(x_vals, kde_pdf, color='black', label=f'KDE (Bandwidth = {round(bandwidth,2)})')
    # puntos de las intersecciones
    x_vals_puntos = np.linspace(int(min(raw_data)), int(max(raw_data)), int(max(raw_data)) - int(min(raw_data)) + 1)
    log_dens_puntos = kde.score_samples(x_vals_puntos[:, None])
    kde_pdf_puntos = np.exp(log_dens_puntos)
    plt.scatter(x_vals_puntos, kde_pdf_puntos, color='red', label=f'KDE intersecciones')


    # Leyendas y nombres
    plt.xlabel('LOS')
    plt.ylabel('Densidad')
    plt.title('Histograma y KDE discreto de LOS')
    plt.legend()
    plt.grid(True)
    plt.show()
    ##############################################################

metricas, _ = calculate_kde_metrics(los, ocurrencias, kde)
display(pd.DataFrame(metricas))

In [ ]:
def find_satisfactory_h(data, empirical_counts, support, a1=1, a2=1, initial_h=0.5, p_threshold=0.065):
    p = 0
    h = initial_h
    while p <= p_threshold:
        _, kde = discrete_DT_kde(data, a1=a1, a2=a2, h=h, support=support)
        expected_counts = kde * empirical_counts.sum()
        chi2, p = chisquare(empirical_counts, expected_counts)
        if p > p_threshold:
            return h, kde, chi2, p
        else:
            h -= 0.01
        if h < 0.02:
            print("No satisfactory h found.")
            return None, None, None, None
            # return h, kde, chi2, p

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chisquare

# Datos de ejemplo (reemplaza con tus propios datos si es necesario)
data = {
    "LOS": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
    "count": [486, 1086, 685, 392, 190, 105, 56, 52, 18, 18, 10, 3, 3, 1, 1, 1, 2, 1]
}
df = pd.DataFrame(data)
los_samples = np.repeat(df["LOS"], df["count"])  # Expande los datos a una lista de muestras

# Implementación de KDE discreto según Hirukawa (2010)
def hirukawa_discrete_kde(data, bandwidth=1, min_val=None, max_val=None):
    if min_val is None:
        min_val = np.min(data)
    if max_val is None:
        max_val = np.max(data)

    values = np.arange(min_val, max_val + 1)  # Soporte discreto
    pmf = np.zeros_like(values, dtype=float)  # Inicializa la función de masa de probabilidad

    for x in data:
        # Calcula los pesos del kernel Gaussiano centrado en x
        raw_weights = np.exp(-0.5 * ((values - x) / bandwidth) ** 2)
        # Normaliza los pesos del kernel para que sumen 1 en cada iteración
        normalized_weights = raw_weights / raw_weights.sum()
        pmf += normalized_weights  # Acumula los pesos en la PMF

    pmf /= pmf.sum()  # Normaliza la PMF total para que sume 1
    return values, pmf

# Calcula la PMF empírica a partir de los datos
values = np.arange(1, 19)
empirical_counts = df.set_index("LOS").reindex(values, fill_value=0)["count"].values
empirical_pmf = empirical_counts / empirical_counts.sum()

# Búsqueda de ancho de banda que cumpla con p > 0.065
bandwidths = np.linspace(0.1, 2, 100)
satisfactory_bandwidth = None

for bw in bandwidths[::-1]:
    _, test_pmf = hirukawa_discrete_kde(los_samples, bandwidth=bw, min_val=1, max_val=18)
    expected_counts = test_pmf * empirical_counts.sum()  # Escala la PMF para chi-cuadrado
    chi2, p_val = chisquare(f_obs=empirical_counts, f_exp=expected_counts)
    if p_val > 0.065:
        satisfactory_bandwidth = bw
        break  # Detiene la búsqueda al encontrar el primer ancho válido

# Si se encuentra un KDE satisfactorio, mostrar el gráfico comparativo
if satisfactory_bandwidth is not None:
    values_kde, kde_pmf = hirukawa_discrete_kde(los_samples, bandwidth=satisfactory_bandwidth, min_val=1, max_val=18)

    # Gráfico de comparación entre PMF empírica y KDE ajustado
    plt.figure(figsize=(10, 6))
    plt.bar(values - 0.2, empirical_pmf, width=0.4, label='PMF Empírica', alpha=0.7)
    plt.bar(values + 0.2, kde_pmf, width=0.4, label=f'KDE Hirukawa (bw={satisfactory_bandwidth:.2f})', alpha=0.7)
    plt.title("Comparación: KDE Discreto de Hirukawa vs PMF Empírica (p > 0.065)")
    plt.xlabel("Longitud de Estancia (días)")
    plt.ylabel("Probabilidad")
    plt.xticks(values)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Mostrar resultados del test chi-cuadrado final
    chi2_final, p_val_final = chisquare(empirical_counts, kde_pmf * empirical_counts.sum())
    print(f"Mejor ancho de banda (p > 0.065): {satisfactory_bandwidth:.2f}")
    print(f"Estadístico chi-cuadrado: {chi2_final:.4f}")
    print(f"p-valor: {p_val_final:.4f}")
else:
    print("No se encontró un ancho de banda que cumpla con p > 0.065.")

In [ ]:
# Puntos de intersección entre data y kde (en los enteros: 1, 2, 3 ...)
x_vals = np.linspace(int(min(los)), int(max(los)), int(max(los)))
log_dens = kde.score_samples(x_vals[:, None])
kde_pdf = np.exp(log_dens)

# Almaceno el los y sus ocurrencias en un diccionario
dict_temporal = {}
for i in range(len(los)):
    dict_temporal[int(los[i])] = ocurrencias[i]

# Genero ocurrencias nuevamente pero agregando un cero en los intervalos sin ocurrencias
ocurrencias_con_cero = []
for i in range(1, int(max(los)) + 1):
    ocurrencias_con_cero.append(dict_temporal.get(i, 0))
ocurrencias_con_cero = np.array(ocurrencias_con_cero)

# Calculo las métricas para el kde (que es el unico que voy a usar)
def get_metrics(y_true, y_pred):
    prediccion = y_pred * (y_true.sum() / y_pred.sum()) # reescala predicciones para el test
    chi2 = chisquare(f_obs=y_true, f_exp=prediccion)
    ks_stat = ks_2samp(y_true, y_pred)
    cc= pearsonr(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae = mean_absolute_error(y_true, y_pred)

    return {
        'Chi2': {"Value": round(chi2.statistic, 3), "p-value": round(chi2.pvalue, 3)},
        'KS': {"Value": round(ks_stat.statistic, 3), "p-value": round(ks_stat.pvalue, 3)},
        'CC': {"Value": round(cc.statistic, 3), "p-value": round(cc.pvalue, 3)},
        'R2': {"Value": round(r2, 3), "p-value": "NA"},
        'RMSE': {"Value": round(rmse, 3), "p-value": "NA"},
        'MAE': {"Value": round(mae, 3), "p-value": "NA"}
    }

# Llamo a la función, se utilizan ocurrencias, no probabilidades
metrics = get_metrics(ocurrencias_con_cero, kde_pdf*ocurrencias.sum())

# Se muestran las metricas
display(pd.DataFrame(metrics))